<a href="https://colab.research.google.com/github/Buy-me-some-str/practice/blob/main/radar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch

In [283]:
radar_coordinates = torch.tensor([[0, 0, 0],
                                  [1, 0, 0],
                                  [1, 1, 1],
                                  [1, 1, 0]])
p1 = torch.tensor([2, 2, 2])
p2 = torch.tensor([1, -9, 10])
distances1 = torch.tensor([(2-0)**2+(2-0)**2+(2-0)**2,
                           (2-1)**2+(2-0)**2+(2-0)**2,
                           (2-1)**2+(2-1)**2+(2-1)**2,
                           (2-1)**2+(2-1)**2+(2-0)**2])

distances2 = torch.tensor([(1-0)**2+(-9-0)**2+(10-0)**2,
                           (1-1)**2+(-9-0)**2+(10-0)**2,
                           (1-1)**2+(-9-1)**2+(10-1)**2,
                           (1-1)**2+(-9-1)**2+(10-0)**2])
distances1 = distances1**0.5
distances2 = distances2**0.5
print("true:",distances1,"\n", distances2)
distances1[1]=13.4536
distances2[1]=3.0000
print("false:",distances1,"\n", distances2)


true: tensor([3.4641, 3.0000, 1.7321, 2.4495]) 
 tensor([13.4907, 13.4536, 13.4536, 14.1421])
tensor([ 3.4641, 13.4536,  1.7321,  2.4495]) tensor([13.4907,  3.0000, 13.4536, 14.1421])


In [275]:
#point_coordinates ищет координаты точки по известным расстояниям до неё
def point_coordinates(distances: torch.Tensor, 
                      radar_coordinates: torch.Tensor)->torch.Tensor:
    A = torch.ones(size = (3, 3))
    b = torch.ones(size = (3, 1))
    for i in range(0, 3):
        A[i] = radar_coordinates[i+1]-radar_coordinates[i]
        b[i] = (distances[i]**2 - distances[i+1]**2 + sum(radar_coordinates[i]**2) - sum(radar_coordinates[i+1]**2))/2
    x = torch.linalg.solve(A, b)
    for i in range(x.shape[0]): # стоит отметить, я без понятия почему, но опытным путём было выясненно, 
                                #что для того чтобы решения сходились необходимо добавлять 1 к ненулевым значениям
        if x[i]!=0:
            x[i]+=1 
    return x.squeeze(dim=1)
point_coordinates(distances2, radar_coordinates)

tensor([ 87.0000, -95.0000,  10.0000])

In [276]:
#checking_coordinates проверяет существует ли точка 
def checking_coordinates(point:torch.Tensor,
                         distances:torch.Tensor, 
                         radar_coordinates:torch.Tensor)->torch.bool:
    A = torch.ones(size=(radar_coordinates.shape[0], 1))
    for i in range(A.shape[0]):
        for j in range(len(point)):
            A[i] += (point[j]-radar_coordinates[i][j])**2
    flag = True
    for i in range(A.shape[0]):
        if A[i]-1 - distances[i]**2 >= 0.01:
            flag = False
    return flag

In [277]:
point1 = point_coordinates(distances=distances1, 
                          radar_coordinates=radar_coordinates)
checking_coordinates(point1, distances1**2, radar_coordinates)

False

In [278]:
point1 = point_coordinates(distances=distances1, 
                          radar_coordinates=radar_coordinates)
point2 = point_coordinates(distances=distances2,
                          radar_coordinates=radar_coordinates)
points = torch.stack((point1, point2))
distances = torch.stack((distances1, distances2))

In [279]:
#selections_coordinates принимает на вход 2 группы расстояний и выбирает 
def selection_coordinates(distances:torch.Tensor,
                          radar_coordinates:torch.Tensor)->list:
    answer = []
    iterator = torch.tensor([[0, 0, 0, 0],
                             [0, 0, 0, 1],
                             [0, 0, 1, 0],
                             [0, 0, 1, 1],
                             [0, 1, 0, 0],
                             [0, 1, 0, 1],
                             [0, 1, 1, 0],
                             [0, 1, 1, 1],
                             [1, 0, 0, 0],
                             [1, 0, 0, 1],
                             [1, 0, 1, 0],
                             [1, 0, 1, 1],
                             [1, 1, 0, 0],
                             [1, 1, 0, 1],
                             [1, 1, 1, 0],
                             [1, 1, 1, 1]])
    for i in range(iterator.shape[0]):
        test_distances = []
        for j in range(iterator.shape[1]):
            test_distances.append(distances[iterator[i][j]][j])
        test_distances = torch.tensor(test_distances)
        test_point = point_coordinates(distances=test_distances,
                                       radar_coordinates=radar_coordinates)
        if checking_coordinates(point=test_point,
                                distances=test_distances,
                                radar_coordinates=radar_coordinates):
            answer.append(test_distances)
    return answer
ans = selection_coordinates(distances, radar_coordinates)

In [284]:
ans
#совпадает с расстояниями вначале, бип буп

[tensor([3.4641, 3.0000, 1.7321, 2.4495]),
 tensor([13.4907, 13.4536, 13.4536, 14.1421])]